In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '' 
import jax

jax.config.update('jax_enable_x64', True)
jax.config.update("jax_default_matmul_precision", "highest")

from mace.calculators import mace_mp,mace_off
from ase import build
from fennol import FENNIX
import yaml
import jax.numpy as jnp
import numpy as np  
from fennol.ase import FENNIXCalculator
from fennol.utils import AtomicUnits as au
from copy import deepcopy
from mace import modules as torch_modules
import e3nn as e3nn_torch
import torch
from fennol.utils.activations import normalize_activation

from ase.units import Hartree
print(au.EV)
print(Hartree)

e3nn_torch.__version__

27.211386024367243
27.211386024367243


'0.4.4'

In [2]:
atoms = build.molecule('H2O')
model_name = "mace_mp_small"
atoms.set_positions([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0]])
calc = mace_mp(model="small", dispersion=False, default_dtype="float64", device='cpu')
atoms.calc = calc
# print(atoms.get_potential_energy())

torch_model_mp = calc.models[0].cpu()
species_order = torch_model_mp.atomic_numbers.tolist()
print("species_order:",species_order)
print(max(species_order))
num_species = len(species_order)
print(num_species)

scale = torch_model_mp.scale_shift.scale.item()
shift = torch_model_mp.scale_shift.shift.item()
print("scale",scale/au.EV)
print("shift",shift/au.EV)

avg_num_neigh = torch_model_mp.interactions[0].avg_num_neighbors
print("avg_num_neighbors",avg_num_neigh)
print(torch_model_mp)



Using Materials Project MACE for MACECalculator with /home/pthomas/.cache/mace/46jrkm3v
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
species_order: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 89, 90, 91, 92, 93, 94]
94
89
scale 0.029552110088317672
shift 0.006030452232198983
avg_num_neighbors 61.964672446250916
ScaleShiftMACE(
  (node_embedding): LinearNodeEmbeddingBlock(
    (linear): Linear(89x0e -> 128x0e | 11392 weights)
  )
  (radial_embedding): RadialEmbeddingBlock(
    (bessel_fn): BesselBasis(r_max=6.0, num_basis=10, trainable=False)
    (cutoff_fn): PolynomialCutoff(p=5.0, r_max=6.0)
  )
  (spherical_harmonics): SphericalHarmonics()
  (atom

In [3]:
r_max = torch_model_mp.radial_embedding.bessel_fn.r_max.item()
num_basis = torch_model_mp.radial_embedding.bessel_fn.bessel_weights.numel()
p = torch_model_mp.radial_embedding.cutoff_fn.p.item()
max_ell = torch_model_mp.spherical_harmonics._lmax
num_inter = len(torch_model_mp.products)
corr = len(torch_model_mp.products[0].symmetric_contractions.contractions[0].weights)+1
print(torch_model_mp.readouts[1].non_linearity.acts[0].cst)
print(f"{r_max=},{num_basis=},{p=},{max_ell=},{num_inter=},{num_species=},{avg_num_neigh=},{corr=}")
torch_model = torch_modules.ScaleShiftMACE(
    atomic_inter_scale=scale,
    atomic_inter_shift=shift,
    r_max=r_max,
    num_bessel=num_basis,
    num_polynomial_cutoff=p,
    max_ell=max_ell,
    interaction_cls_first=torch_modules.RealAgnosticResidualInteractionBlock,
    interaction_cls=torch_modules.RealAgnosticResidualInteractionBlock,
    num_interactions=num_inter,
    num_elements=num_species,
    hidden_irreps=e3nn_torch.o3.Irreps("128x0e"),
    MLP_irreps="16x0e",
    avg_num_neighbors=avg_num_neigh,
    correlation=corr,
    atomic_energies=torch_model_mp.atomic_energies_fn.atomic_energies.clone().detach(),
    atomic_numbers=torch_model_mp.atomic_numbers.clone().detach(),
    gate=torch.nn.SiLU(),
).double()
print(torch_model)

1.6791767923989418
r_max=6.0,num_basis=10,p=5.0,max_ell=3,num_inter=2,num_species=89,avg_num_neigh=61.964672446250916,corr=3


/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/mace/modules/models.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "atomic_numbers", torch.tensor(atomic_numbers, dtype=torch.int64)
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/mace/modules/blocks.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(atomic_

ScaleShiftMACE(
  (node_embedding): LinearNodeEmbeddingBlock(
    (linear): Linear(89x0e -> 128x0e | 11392 weights)
  )
  (radial_embedding): RadialEmbeddingBlock(
    (bessel_fn): BesselBasis(r_max=6.0, num_basis=10, trainable=False)
    (cutoff_fn): PolynomialCutoff(p=5.0, r_max=6.0)
  )
  (spherical_harmonics): SphericalHarmonics()
  (atomic_energies_fn): AtomicEnergiesBlock(energies=[-3.6672, -1.3321, -3.4821, -4.7367, -7.7249, -8.4056, -7.3601, -7.2846, -4.8965, 0.0000, -2.7594, -2.8140, -4.8469, -7.6948, -6.9633, -4.6726, -2.8117, -0.0626, -2.6176, -5.3905, -7.8858, -10.2684, -8.6651, -9.2331, -8.3050, -7.0490, -5.5774, -5.1727, -3.2521, -1.2902, -3.5271, -4.7085, -3.9765, -3.8862, -2.5185, 6.7669, -2.5635, -4.9380, -10.1498, -11.8469, -12.1389, -8.7917, -8.7869, -7.7809, -6.8500, -4.8910, -2.0634, -0.6396, -2.7887, -3.8186, -3.5871, -2.8804, -1.6356, 9.8467, -2.7653, -4.9910, -8.9337, -8.7356, -8.0190, -8.2515, -7.5917, -8.1697, -13.5927, -18.5175, -7.6474, -8.1230, -7.6078, -6.

/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.w

In [4]:
torch_model.load_state_dict(torch_model_mp.state_dict())
calc = mace_mp(model="small", dispersion=False, default_dtype="float64", device='cpu')
calc.models[0] = torch_model
atoms.calc = calc
print(calc.get_potential_energy(atoms,force_consistent=True))

Using Materials Project MACE for MACECalculator with /home/pthomas/.cache/mace/46jrkm3v
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
-13.883318032136243


In [5]:
print(torch_model.products[0].symmetric_contractions.contractions[0].correlation)


3


In [6]:
model_params = yaml.load(open(model_name+".yaml"), Loader=yaml.FullLoader)
model_fennol = FENNIX(**model_params,rng_key=jax.random.PRNGKey(1)) 

In [7]:
variables = deepcopy(model_fennol.variables)
nn_energy_params = variables["params"]["nn_energy"]

for k in nn_energy_params.keys():
    print(k)
    if isinstance(nn_energy_params[k],dict):
        for kk in nn_energy_params[k].keys():
            print("   ",kk)


SymmetricContraction_0
    w1_0e
    w2_0e
    w3_0e
SymmetricContraction_1
    w1_0e
    w2_0e
    w3_0e
hidden_linear_readout_last
    w[0,0] 128x0e,16x0e
linear_contraction_0
    w[0,0] 128x0e,128x0e
linear_contraction_1
    w[0,0] 128x0e,128x0e
linear_dn_0
    w[0,0] 128x0e,128x0e
    w[1,1] 128x1o,128x1o
    w[2,2] 128x2e,128x2e
    w[3,3] 128x3o,128x3o
linear_dn_1
    w[0,0] 128x0e,128x0e
    w[1,1] 128x1o,128x1o
    w[2,2] 128x2e,128x2e
    w[3,3] 128x3o,128x3o
linear_readout_0
    w[0,0] 128x0e,1x0e
linear_readout_last
    w[0,0] 16x0e,1x0e
linear_up_0
    w[0,0] 128x0e,128x0e
linear_up_1
    w[0,0] 128x0e,128x0e
radial_network_0
    Dense_0
    Dense_1
    Dense_2
    Dense_3
radial_network_1
    Dense_0
    Dense_1
    Dense_2
    Dense_3
skip_tp_0
    w[0,0] 128x0e,128x0e
skip_tp_1
    w[0,0] 128x0e,128x0e
species_encoding


In [8]:
def linear_torch_to_jax(linear):
    return {
        f"w[{ins.i_in},{ins.i_out}] {linear.irreps_in[ins.i_in]},{linear.irreps_out[ins.i_out]}": jnp.asarray(
            w.data
        )
        for i, ins, w in linear.weight_views(yield_instruction=True)
    }

def skip_tp_torch_to_jax(tp):
    return {
        f"w[{ins.i_in1},{ins.i_out}] {tp.irreps_in1[ins.i_in1]},{tp.irreps_out[ins.i_out]}": jnp.moveaxis(
            jnp.asarray(w.data), 1, 0
        )/num_species**0.5
        for i, ins, w in tp.weight_views(yield_instruction=True)
    }

In [9]:
## get initial species encodings
species_encodings = np.zeros(nn_energy_params["species_encoding"].shape,dtype=np.float64)
species_encodings_torch = torch_model.node_embedding.linear.weight.detach().numpy().reshape(num_species,-1)
assert species_encodings.shape[1] == species_encodings_torch.shape[1]
factor = num_species**-0.5
for i,z in enumerate(species_order):
    species_encodings[z,:] = species_encodings_torch[i,:]*factor
    # print(f"{z}: {species_encodings[z]}")
            
nn_energy_params["species_encoding"] = jnp.array(species_encodings,dtype=jnp.float64)
print(species_encodings[1,:10])
print(species_encodings_torch[0,:10])
print(torch_model.node_embedding.linear.instructions)

[-0.09600671 -0.04891878 -0.12642812 -0.00084473 -0.02929069 -0.00255149
  0.09475341  0.1371028  -0.03754604 -0.02659503]
[-0.9057255  -0.46149882 -1.1927205  -0.00796918 -0.27632779 -0.02407071
  0.89390186  1.29342525 -0.35420867 -0.25089705]
[Instruction(i_in=0, i_out=0, path_shape=(89, 128), path_weight=0.105999788000636)]


In [10]:
# species energy shift

print(len(model_fennol.variables["params"]["energy_shift"]["constant"]))
print(len(torch_model.atomic_energies_fn.atomic_energies))
energy_shift = np.zeros(len(model_fennol.variables["params"]["energy_shift"]["constant"]),dtype=np.float64)
for i,z in enumerate(species_order):
    # print(f"{z}: {model_fennol.variables['params']['energy_shift']['constant'][z]} {torch_model.atomic_energies_fn.atomic_energies[i].data/au.EV}")
    energy_shift[z] = torch_model.atomic_energies_fn.atomic_energies[i].data/au.EV
variables["params"]["energy_shift"]["constant"] = jnp.array(energy_shift,dtype=jnp.float64)
print(variables["params"]["energy_shift"]["constant"])
    

119
89
[ 0.00000000e+00 -1.34765940e-01 -4.89536002e-02 -1.27964837e-01
 -1.74070414e-01 -2.83886143e-01 -3.08899133e-01 -2.70478705e-01
 -2.67704064e-01 -1.79942722e-01  5.11468097e-14 -1.01404660e-01
 -1.03414343e-01 -1.78119602e-01 -2.82778434e-01 -2.55896403e-01
 -1.71716001e-01 -1.03327676e-01 -2.30032546e-03 -9.61966981e-02
 -1.98095792e-01 -2.89797631e-01 -3.77356485e-01 -3.18438310e-01
 -3.39308360e-01 -3.05201342e-01 -2.59045481e-01 -2.04967133e-01
 -1.90094970e-01 -1.19511468e-01 -4.74125486e-02 -1.29617881e-01
 -1.73032698e-01 -1.46134081e-01 -1.42816358e-01 -9.25529485e-02
  2.48680741e-01 -9.42067374e-02 -1.81468346e-01 -3.72998966e-01
 -4.35363990e-01 -4.46096217e-01 -3.23088239e-01 -3.22914437e-01
 -2.85943987e-01 -2.51733646e-01 -1.79741637e-01 -7.58296426e-02
 -2.35037477e-02 -1.02484460e-01 -1.40331120e-01 -1.31822331e-01
 -1.05852917e-01 -6.01071435e-02  3.61860430e-01 -1.01622332e-01
 -1.83414267e-01 -3.28306864e-01 -3.21027057e-01 -2.94691568e-01
 -3.03236739e-01 -

In [11]:
# skip tp 0
for layer in range(2):
    print(f'skip_tp_{layer}')
    skip_params_torch = skip_tp_torch_to_jax(
        torch_model.interactions[layer].skip_tp
    )
    print(skip_params_torch.keys())
    print(nn_energy_params[f"skip_tp_{layer}"].keys())
    for k in nn_energy_params[f'skip_tp_{layer}'].keys():
        print(skip_params_torch[k].shape)
        print(nn_energy_params[f'skip_tp_{layer}'][k].shape)
        skip_params = np.zeros_like(nn_energy_params[f'skip_tp_{layer}'][k])
        assert skip_params.shape[1:] == nn_energy_params[f'skip_tp_{layer}'][k].shape[1:]
        for i,z in enumerate(species_order):
            skip_params[z,:,:] = np.array(skip_params_torch[k][i,:,:])

        nn_energy_params[f'skip_tp_{layer}'][k] = jnp.array(skip_params,dtype=jnp.float64)

# # skip tp 1
# print()
# print("skip tp 1")
# skip_params_torch = skip_tp_torch_to_jax(
#         torch_model.interactions[1].skip_tp
#     )
# print(skip_params_torch.keys())
# print(nn_energy_params["skip_tp_1"].keys())

# print(skip_params_torch['w[0,0] 128x0e,128x0e'].shape)
# print(nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'].shape)
# skip_params = np.zeros_like(nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'])
# assert skip_params.shape[1:] == nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'].shape[1:]
# for i,z in enumerate(species_order):
#     skip_params[z,:,:] = np.array(skip_params_torch['w[0,0] 128x0e,128x0e'][i,:,:])

# nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'] = jnp.array(skip_params,dtype=jnp.float64)

skip_tp_0
dict_keys(['w[0,0] 128x0e,128x0e'])
dict_keys(['w[0,0] 128x0e,128x0e'])
(89, 128, 128)
(120, 128, 128)
skip_tp_1
dict_keys(['w[0,0] 128x0e,128x0e'])
dict_keys(['w[0,0] 128x0e,128x0e'])
(89, 128, 128)
(120, 128, 128)


In [12]:
# linear up
for i in range(2):
    print()
    print(f"linear_up_{i}")
    up_params_torch = linear_torch_to_jax(torch_model.interactions[i].linear_up)
    print(up_params_torch.keys())
    print(nn_energy_params[f"linear_up_{i}"].keys())
    for k in up_params_torch.keys():
        print(k)
        print(up_params_torch[k].shape)
        print(nn_energy_params[f"linear_up_{i}"][k].shape)
        assert up_params_torch[k].shape == nn_energy_params[f"linear_up_{i}"][k].shape
        nn_energy_params[f"linear_up_{i}"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)

# linear down
for i in range(2):
    print()
    print(f"linear_dn_{i}")
    up_params_torch = linear_torch_to_jax(torch_model.interactions[i].linear)
    print(up_params_torch.keys())
    print(nn_energy_params[f"linear_dn_{i}"].keys())
    for k in up_params_torch.keys():
        print(k)
        print(up_params_torch[k].shape)
        print(nn_energy_params[f"linear_dn_{i}"][k].shape)
        assert up_params_torch[k].shape == nn_energy_params[f"linear_dn_{i}"][k].shape
        nn_energy_params[f"linear_dn_{i}"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)

# contraction linear
for i in range(2):
    print()
    print(f"linear_contraction_{i}")
    up_params_torch = linear_torch_to_jax(torch_model.products[i].linear)
    print(up_params_torch.keys())
    print(nn_energy_params[f"linear_contraction_{i}"].keys())
    for k in up_params_torch.keys():
        print(k)
        print(up_params_torch[k].shape)
        try:
            print(nn_energy_params[f"linear_contraction_{i}"][k].shape)
            assert up_params_torch[k].shape == nn_energy_params[f"linear_contraction_{i}"][k].shape
            nn_energy_params[f"linear_contraction_{i}"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)
        except KeyError:
            print(f"key {k} not found in nn_energy_params[f'linear_contraction_{i}']")



linear_up_0
dict_keys(['w[0,0] 128x0e,128x0e'])
dict_keys(['w[0,0] 128x0e,128x0e'])
w[0,0] 128x0e,128x0e
(128, 128)
(128, 128)

linear_up_1
dict_keys(['w[0,0] 128x0e,128x0e'])
dict_keys(['w[0,0] 128x0e,128x0e'])
w[0,0] 128x0e,128x0e
(128, 128)
(128, 128)

linear_dn_0
dict_keys(['w[0,0] 128x0e,128x0e', 'w[1,1] 128x1o,128x1o', 'w[2,2] 128x2e,128x2e', 'w[3,3] 128x3o,128x3o'])
dict_keys(['w[0,0] 128x0e,128x0e', 'w[1,1] 128x1o,128x1o', 'w[2,2] 128x2e,128x2e', 'w[3,3] 128x3o,128x3o'])
w[0,0] 128x0e,128x0e
(128, 128)
(128, 128)
w[1,1] 128x1o,128x1o
(128, 128)
(128, 128)
w[2,2] 128x2e,128x2e
(128, 128)
(128, 128)
w[3,3] 128x3o,128x3o
(128, 128)
(128, 128)

linear_dn_1
dict_keys(['w[0,0] 128x0e,128x0e', 'w[1,1] 128x1o,128x1o', 'w[2,2] 128x2e,128x2e', 'w[3,3] 128x3o,128x3o'])
dict_keys(['w[0,0] 128x0e,128x0e', 'w[1,1] 128x1o,128x1o', 'w[2,2] 128x2e,128x2e', 'w[3,3] 128x3o,128x3o'])
w[0,0] 128x0e,128x0e
(128, 128)
(128, 128)
w[1,1] 128x1o,128x1o
(128, 128)
(128, 128)
w[2,2] 128x2e,128x2e
(128, 1

In [13]:
# readout block
print(f"linear_readout_0")
up_params_torch = linear_torch_to_jax(torch_model.readouts[0].linear)
print(up_params_torch.keys())
print(nn_energy_params[f"linear_readout_0"].keys())
for k in up_params_torch.keys():
    print(k)
    print(up_params_torch[k].shape)
    print(nn_energy_params[f"linear_readout_0"][k].shape)
    assert up_params_torch[k].shape == nn_energy_params[f"linear_readout_0"][k].shape
    nn_energy_params[f"linear_readout_0"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)

print()
print(f"hidden_linear_readout_last")
up_params_torch = linear_torch_to_jax(torch_model.readouts[1].linear_1)
print(up_params_torch.keys())
print(nn_energy_params[f"hidden_linear_readout_last"].keys())
for k in up_params_torch.keys():
    print(k)
    print(up_params_torch[k].shape)
    print(nn_energy_params[f"hidden_linear_readout_last"][k].shape)
    assert up_params_torch[k].shape == nn_energy_params[f"hidden_linear_readout_last"][k].shape
    nn_energy_params[f"hidden_linear_readout_last"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)


f,silu_norm = normalize_activation(jax.nn.silu,return_scale=True)
print("silu_norm",silu_norm,1.6765647703084872)
cst = torch_model.readouts[1].non_linearity.acts[0].cst / silu_norm

print()
print(f"linear_readout_last")
up_params_torch = linear_torch_to_jax(torch_model.readouts[1].linear_2)
print(up_params_torch.keys())
print(nn_energy_params[f"linear_readout_last"].keys())
for k in up_params_torch.keys():
    print(k)
    print(up_params_torch[k].shape)
    print(nn_energy_params[f"linear_readout_last"][k].shape)
    assert up_params_torch[k].shape == nn_energy_params[f"linear_readout_last"][k].shape
    nn_energy_params[f"linear_readout_last"][k] = cst*jnp.array(up_params_torch[k],dtype=jnp.float64)

print(torch_model.readouts[1].non_linearity.acts[0].cst)


linear_readout_0
dict_keys(['w[0,0] 128x0e,1x0e'])
dict_keys(['w[0,0] 128x0e,1x0e'])
w[0,0] 128x0e,1x0e
(128, 1)
(128, 1)

hidden_linear_readout_last
dict_keys(['w[0,0] 128x0e,16x0e'])
dict_keys(['w[0,0] 128x0e,16x0e'])
w[0,0] 128x0e,16x0e
(128, 16)
(128, 16)
silu_norm 1.6765620560798837 1.6765647703084872

linear_readout_last
dict_keys(['w[0,0] 16x0e,1x0e'])
dict_keys(['w[0,0] 16x0e,1x0e'])
w[0,0] 16x0e,1x0e
(16, 1)
(16, 1)
1.6791767923989418


In [14]:
# radial net

for i in range(2):
    print("radial net",i)
    print(nn_energy_params[f"radial_network_{i}"].keys())
    w  = torch_model.interactions[i].conv_tp_weights.layer0.weight.detach().numpy()
    w = w/w.shape[0]**0.5
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_0"]["kernel"].shape)
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_0"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_0"]["kernel"] = jnp.array(w,dtype=jnp.float64)

    cst = torch_model.interactions[i].conv_tp_weights.layer0.act.cst/ silu_norm
    print("cst",cst)

    w  = cst*torch_model.interactions[i].conv_tp_weights.layer1.weight.detach().numpy()
    w = w/w.shape[0]**0.5
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_1"]["kernel"].shape)
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_1"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_1"]["kernel"] =  jnp.array(w,dtype=jnp.float64)

    cst = torch_model.interactions[i].conv_tp_weights.layer0.act.cst/ silu_norm
    print("cst",cst)

    w  = cst*torch_model.interactions[i].conv_tp_weights.layer2.weight.detach().numpy()
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_2"]["kernel"].shape)
    w = w/w.shape[0]**0.5
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_2"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_2"]["kernel"] =  jnp.array(w,dtype=jnp.float64)

    cst = torch_model.interactions[i].conv_tp_weights.layer0.act.cst/ silu_norm
    print("cst",cst)

    w  = cst*torch_model.interactions[i].conv_tp_weights.layer3.weight.detach().numpy()
    w = w/w.shape[0]**0.5
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_3"]["kernel"].shape)
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_3"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_3"]["kernel"] =  jnp.array(w,dtype=jnp.float64)
    print()


radial net 0
dict_keys(['Dense_0', 'Dense_1', 'Dense_2', 'Dense_3'])
(10, 64) (10, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 512) (64, 512)

radial net 1
dict_keys(['Dense_0', 'Dense_1', 'Dense_2', 'Dense_3'])
(10, 64) (10, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 512) (64, 512)



In [15]:
from mace.tools.cg import U_matrix_real
from e3nn_jax import reduced_tensor_product_basis
from e3nn import o3
import e3nn_jax

def get_reordering(order,ir_out):
    # torch 
    irreps_in = o3.Irreps("1x0e + 1x1o + 1x2e + 1x3o")
    irrep_out = o3.Irreps("1x"+ir_out)
    U_matrix = U_matrix_real(irreps_in=irreps_in, irreps_out=irrep_out, correlation=order)[-1].numpy()
    U_matrix_abs = np.abs(U_matrix)

    # jax
    irreps_in = e3nn_jax.Irreps("1x0e + 1x1o + 1x2e + 1x3o")
    irrep_out = e3nn_jax.Irrep(ir_out)
    U = reduced_tensor_product_basis([irreps_in] * order, keep_ir=[irrep_out]).list[-1]
    U = np.moveaxis(U,-1,0)
    U_abs = np.abs(U)

    index = []
    signs = []
    for i in range(U.shape[-1]):
        diffs = []
        for k in range(U_matrix.shape[-1]):
            diff = np.max(np.abs(U_abs[...,i]-U_matrix_abs[...,k]))
            diffs.append(diff)
        k = np.argmin(diffs)
        diff = diffs[k]
        if diff>1.e-6:
            print(i,k,diff)
            raise ValueError("diff too large")
        
        index.append(k)
        diff = np.max(np.abs(U[...,i]-U_matrix[...,k]))
        if diff < 1.e-6:
            signs.append(1.)
        else:
            diff = np.max(np.abs(U[...,i]+U_matrix[...,k]))
            if diff < 1.e-6:
                print(i,k,"global sign differ")
                signs.append(-1.)
            else:
                raise ValueError("local sign differ")
    print(order,ir_out,len(index),U.shape[-1],len(signs))
    return np.array(index),np.array(signs)
    return np.arange(U.shape[-1])

In [16]:
ir_layers = [["0e"],["0e"]]
for layer in [0,1]:
    print()
    print("layer",layer,"->",list(nn_energy_params[f"SymmetricContraction_{layer}"].keys()))

    for ir in ir_layers[layer]:
        l=int(ir[0])
        corr = len(torch_model.products[layer].symmetric_contractions.contractions[l].weights)+1
        print("   ir",ir,"l",l,"corr",corr)

        w=torch_model.products[layer].symmetric_contractions.contractions[l].weights_max.detach().cpu().numpy()

        print(f"     w{corr}_{ir}",w.shape,nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"].shape) 
        assert w.shape[1:] == nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"].shape[1:]

        # reorder weights
        index,signs = get_reordering(corr,ir)
        w = w[:,index,:]*signs[None,:,None]


        # last order
        skip_params = np.zeros_like(nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"])
        for i,z in enumerate(species_order):
            skip_params[z,:,:] = w[i,:,:]
        nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"] = jnp.asarray(skip_params,dtype=jnp.float64)

        print(np.abs(w).mean())

        for c in range(corr-1):
            order = corr-c-1
            w=torch_model.products[layer].symmetric_contractions.contractions[l].weights[c].detach().cpu().numpy()
            print(f"     w{order}_{ir}",w.shape,nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"].shape)
            assert w.shape[1:] == nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"].shape[1:]

            index,signs = get_reordering(order,ir)
            w = w[:,index,:]*signs[None,:,None]

            skip_params = np.zeros_like(nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"])
            for i,z in enumerate(species_order):
                skip_params[z,:,:] = w[i,:,:]
            nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"] = jnp.asarray(skip_params,dtype=jnp.float64)
            print(np.abs(w).mean())



layer 0 -> ['w1_0e', 'w2_0e', 'w3_0e']
   ir 0e l 0 corr 3
     w3_0e (89, 23, 128) (120, 23, 128)


3 0e 23 23 23
0.12137811837893642
     w2_0e (89, 4, 128) (120, 4, 128)
2 0e 4 4 4
0.27486145801722217
     w1_0e (89, 1, 128) (120, 1, 128)
1 0e 1 1 1
0.468239133582122

layer 1 -> ['w1_0e', 'w2_0e', 'w3_0e']
   ir 0e l 0 corr 3
     w3_0e (89, 23, 128) (120, 23, 128)
3 0e 23 23 23
0.0098936799287686
     w2_0e (89, 4, 128) (120, 4, 128)
2 0e 4 4 4
0.08516793992552044
     w1_0e (89, 1, 128) (120, 1, 128)
1 0e 1 1 1
0.32557432233660294


In [17]:
variables["params"]["nn_energy"] = nn_energy_params  
model_fennol.variables = variables 

model_fennol.save(model_name+".fnx")
model_fennol_load = FENNIX.load(model_name+".fnx")
# print("##############################")
# print("Restored model")
# for k in model_fennol_load.variables["params"]["nn_energy"].keys():
#     print(k)
#     if isinstance(nn_energy_params[k],dict):
#         for kk in nn_energy_params[k].keys():
#             print("   ",kk)
calc_fennol = FENNIXCalculator(model=model_fennol_load,atoms=atoms,use_float64=True)
atoms.calc = calc_fennol
print("MACE fennol",atoms.get_potential_energy())
atoms.calc = calc  
print("MACE torch",atoms.get_potential_energy())


MACE fennol -13.883318033770568
MACE torch -13.883318032136243


In [18]:
species = atoms.get_atomic_numbers()
coordinates = atoms.get_positions()
output = model_fennol_load(coordinates=coordinates,species=species)
print(output.keys())
print(output["nn_energy"]*au.EV)
# print(output["nn_energy_node_feats"][0])

dict_keys(['batch_index', 'coordinates', 'energy_shift', 'graph', 'natoms', 'nn_energy', 'nn_energy_node_feats', 'species'])
[0.13850512 0.29855576 0.29855576]


In [19]:
node_feats_torch = calc.get_descriptors(atoms)
# print(node_feats_torch[0])
print(calc.get_property("node_energy",atoms=atoms))

[0.13850512 0.29855576 0.29855576]
